# process_cmip6_files.ipynb
Post-processing of CMIP6 data that have been downloaded from ESGF:
1. Regrid to regular lon/lat grid (using CDO) and calculate statistics (e.g. CDO's yearmean).

B. S. Grandey, 2022

In [1]:
! date

Mon Apr 11 08:59:01 +08 2022


In [2]:
import contextlib
import pathlib
from cdo import Cdo, CDOException

In [3]:
cdo = Cdo()

print(f'CDO version: {cdo.version()}')
print(f'cdo.py bindings version: {cdo.__version__()}')

CDO version: 2.0.3
cdo.py bindings version: 1.5.4


## Base paths

In [4]:
# CMIP6 input data downloaded from ESGF (organised as <variable>/<source_id>_<member_id>/<instance_id>)
in_base = pathlib.Path('~/Data/p22b/CMIP6/').expanduser()
# Output data
out_base = pathlib.Path('~/Data/p22c/CMIP6/').expanduser()
out_base.mkdir(exist_ok=True)

## Function to regrid available data (using CDO) for a given variable, table_id, source-member pair, and experiment
Notes regarding available interpolation methods, after testing CDO's gencon2, gencon, and genbic on Omon zos data:
1. For unstructured grids, gencon works but not genbic ("Bilinear/bicubic interpolation doesn't support unstructured source grids!").
2. For curvilinear grids, genbic works. gencon/gencon2 mostly works, but fails for some grids (with most common error being "Source grid cell corner coordinates missing!"), and produces strange results for some others.
3. For lonlat grids, all methods work.

Based on the above observations, one possibility would be to prefer genbic for lonlat and curvilinear, and gencon for unstructured. If these fail, then other methods - including gennn (nearest neighbour) - can be used as fallback methods.

In [5]:
def regrid_using_cdo(variable='zos',
                     table_id='Omon',
                     source_member='ACCESS-CM2_r1i1p1f1',
                     experiment='historical',
                     missto0=True,  # setting missing values (and NaNs) to 0 before and after regridding?
                     stat_str='-yearmean',  # calculate statistical values after regridding?
                     force=False,  # overwrite previously produced files?
                    ):
    
    print(f'---- regrid {variable} {table_id} {source_member} {experiment} missto0={missto0} {stat_str} ----')
    
    # List of pre-existing files skipped (assuming force=False)
    existing_list = []
    # List of output files written
    out_list = []
    
    # Search for available input directories, then identify most appropriate directory
    in_dirs = list(in_base.glob(f'{variable}/{source_member}/CMIP6.*.{experiment}.*.{table_id}.{variable}.*'))
    in_dirs = sorted(in_dirs)
    # Ignore directories with no NetCDF files
    for d in in_dirs:
        if len(list(d.glob("*.nc"))) == 0:
            in_dirs.remove(d)
    # Case: no input directories
    if len(in_dirs) == 0:
        print('No suitable input data found')
        return None  # exit (if no input directories)
    # Case: one suitable input directory
    elif len(in_dirs) == 1:
        in_dir = in_dirs[0]
        print(f'1 input dir: {in_dir.name} ({len(list(in_dir.glob("*.nc")))} nc file(s))')
    # Case: more than one suitable input directory
    else:
        print(f'{len(in_dirs)} input dirs:')
        for d in in_dirs:
            print(f'{d.name} ({len(list(d.glob("*.nc")))} nc file(s))')
        # Select most recently dated version
        vmax = 0  # initialise "max" version as 0
        for d in in_dirs:
            vmax = max(vmax, int(d.name[-8:]))
        for d in in_dirs:
            if str(vmax) not in d.name:
                in_dirs.remove(d)
        # After version selection, does only one input dir remain?
        if len(in_dirs) == 1:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (most recent) ')
        # Prefer data that has already been remapped
        else:
            for d in in_dirs:
                if '.gn.' in d.name:
                    in_dirs.remove(d)
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (non-native grid)')
            
    # Is this a special case that requires additional pre-processing?
    # Special case 1: AWI-CM-1-1-MR thetaoga (which CDO fails to read)
    if (source_member.split('_')[0] == 'AWI-CM-1-1-MR') and (variable in ['thetaoga',]):
        print(f'Pre-processing {source_member} {variable} (remove nodes and ncells dims using ncwa)')
        # Output directory for pre-processing (special case 1)
        pre_dir = out_base.joinpath(f'regrid_pre/{variable}/{source_member}/{in_dir.name}')
        pre_dir.mkdir(parents=True, exist_ok=True)
        # Lists of pre-processing files that are skipped (already existing) and of files written
        skipped_pre_list = []
        written_pre_list = []
        # Loop over input data files (special case 1)
        in_fns = sorted(list(in_dir.glob('*.nc')))
        for in_fn in in_fns:
            # Pre-processing filename (special case 1)
            pre_fn = pre_dir.joinpath(f'{in_fn.stem}.ncwa.nc')
            # Do not overwrite, unless force=True (special case 1)
            if pre_fn.exists() and not force:
                skipped_pre_list.append(pre_fn.name)
                continue
            # Use NCO to remove nodes and ncells dimensions (special case 1)
            ! ncwa -a nodes,ncells {in_fn} {pre_fn}
            if pre_fn.exists():
                written_pre_list.append(pre_fn.name)
            else:
                print(f'Failed to write {pre_fn.name}')
        # Print summaries of existing files skipped and files written (special case 1)
        if len(skipped_pre_list) >= 1:
            print(f'Skipped {len(skipped_pre_list)} existing files, inc {skipped_pre_list[0]}')
        if len(written_pre_list) >= 1:
            print(f'Written {len(written_pre_list)} files, inc {written_pre_list[0]}')
        # Input directory is now pre_dir (special case 1)
        in_dir = pre_dir
    # Special case 2: IPSL-CM6A-LR_r1i1p1f1 2D ocean (CDO sees additional generic grid in some files)
    if (source_member == 'IPSL-CM6A-LR_r1i1p1f1') and (variable in ['zos', 'hfds', 'hfgeou', 'hfcorr']):
        print(f'Pre-processing {source_member} {experiment} {variable} (add area:coordinates attribute)')
        # Output directory for pre-processing (special case 2)
        pre_dir = out_base.joinpath(f'regrid_pre/{variable}/{source_member}/{in_dir.name}')
        pre_dir.mkdir(parents=True, exist_ok=True)
        # Lists of pre-processing files that are skipped (already existing) and of files written
        skipped_pre_list = []
        written_pre_list = []
        # Loop over input data files (special case 2)
        in_fns = sorted(list(in_dir.glob('*.nc')))
        for in_fn in in_fns:
            # Pre-processing filename (special case 2)
            pre_fn = pre_dir.joinpath(f'{in_fn.stem}.ncatted.nc')
            # Do not overwrite, unless force=True (special case 2)
            if pre_fn.exists() and not force:
                skipped_pre_list.append(pre_fn.name)
                continue
            # Use NCO to add area:coordinates attribute (special case 2)
            ! ncatted -a coordinates,area,o,c,"nav_lon nav_lat" {in_fn} {pre_fn}
            if pre_fn.exists():
                written_pre_list.append(pre_fn.name)
            else:
                print(f'Failed to write {pre_fn.name}')
        # Print summaries of existing files skipped and files written (special case 2)
        if len(skipped_pre_list) >= 1:
            print(f'Skipped {len(skipped_pre_list)} existing files, inc {skipped_pre_list[0]}')
        if len(written_pre_list) >= 1:
            print(f'Written {len(written_pre_list)} files, inc {written_pre_list[0]}')
        # Input directory is now pre_dir (special case 2)
        in_dir = pre_dir
    
    # Identify input data files
    in_fns = sorted(list(in_dir.glob('*.nc')))
    # Input grid type
    try:
        with contextlib.redirect_stdout(None):  # suppress CDO error message details (if any)
            grid_str = [s for s in cdo.sinfo(input=f'{in_fns[0]}') if 'points' in s][0]
        in_grid_type = grid_str.split(':')[1].strip()
        in_grid_points = grid_str.split(':')[-1].strip()
        print(f'Input data grid type is {in_grid_type} ({in_grid_points})')
    except CDOException:  # if Globus transfer is in progress, incomplete files may be encountered
        print(f'Failed to read {in_fns[0].name} - is it currently being transferred?')
        return None  # exit (if incomplete file encountered)
    # Output data directory
    if stat_str:
        if missto0:
            out_dir = out_base.joinpath(
                f'regrid_missto0_{stat_str.strip("-")}/{variable}/{source_member}/{in_dir.name}')
        else:
            out_dir = out_base.joinpath(f'regrid_{stat_str.strip("-")}/{variable}/{source_member}/{in_dir.name}')
    else:
        if missto0:
            out_dir = out_base.joinpath(f'regrid_missto0/{variable}/{source_member}/{in_dir.name}')
        else:
            out_dir = out_base.joinpath(f'regrid/{variable}/{source_member}/{in_dir.name}')
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # If points=1 (e.g. due to already being a global mean), then just reduce dimensions
    if in_grid_points in ['points=1', 'points=1 (1x1)']:
        print('Reduce dimensions')
        # Loop over in_fns (if points=1)
        for in_fn in in_fns:
            # Output filename (if points=1)
            if stat_str:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.1d.{stat_str.strip("-")}.nc')
            else:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.1d.nc')
            # Do not overwrite, unless force=True (if points=1)
            if out_fn.exists() and not force:
                existing_list.append(out_fn.name)
                continue
            # Write output file (if points=1)
            cdo.selname(variable, input=f'{stat_str} {in_fn}', output=f'{out_fn}',
                        options='--reduce_dim')
            if out_fn.exists():
                out_list.append(out_fn.name)
            else:
                print(f'Failed to write {out_fn.name}')
        # Print summary of existing files skipped (if points=1)
        if len(existing_list) == 1:
            print(f'Skipped existing file: {existing_list[0]}')
        elif len(existing_list) > 1:
            print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
        # Print summary of files written (if points=1)
        if len(out_list) == 1:
            print(f'Written {out_list[0]}')
        elif len(out_list) > 1:
            print(f'Written {len(out_list)} files, inc {out_list[0]}')
        return out_list  # exit (if points=1)
    
    # Preferred regridding methods depend on input grid type
    # Curvilinear: prefer bicubic. (Issues with gencon2 and gencon for some CMIP6 curvilinear grids.)
    # Lonlat, gaussian: same preference as for curvilinear.
    if in_grid_type in ['lonlat', 'curvilinear', 'gaussian']:
        preferred_methods=['genbic', 'gencon', 'gennn', None]
    # Unstructured: prefer 1st order conservative. (Issues with 2nd order conservative for unstructured.)
    elif in_grid_type in ['unstructured',]:
        preferred_methods=['gencon', 'genbic', 'gennn', None]
    else:
        preferred_methods=['gencon', 'genbic', 'gennn', None]
    # Weights data directory
    weights_dir = out_base.joinpath(f'regrid_weights/{variable}/{source_member}/{in_dir.name}')
    weights_dir.mkdir(parents=True, exist_ok=True)
    # Generate regridding weights file
    for method in preferred_methods:
        print(f'Trying {method}')
        weights_fn = weights_dir.joinpath(f'{in_fns[0].stem}.{method}_weights.nc')
        # If all preferred methods have been exhausted, then exit
        if method == None:
            return None  # exit
        # If weights file already exists, use pre-existing file (unless force=True)
        if weights_fn.exists() and not force:
            print(f'{weights_fn.name} already exists')
            break  # if weights file exists, no need to try other methods
        # Otherwise, try calculating weights and writing file
        else:
            try:
                with contextlib.redirect_stdout(None):  # suppress CDO error message details
                    if method == 'gencon2':
                        cdo_res = cdo.gencon2('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gencon':
                        cdo_res = cdo.gencon('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')          
                    elif method == 'genbic':
                        cdo_res = cdo.genbic('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gennn':
                        cdo_res = cdo.gennn('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    else:
                        continue
                if weights_fn.exists():
                    print(f'Written {weights_fn.name}')
                    break  # if file written successfully, then no need to try other methods
            except CDOException:
                print(f'Failed to write {weights_fn.name}')
    
    # Loop over input data files and regrid
    print(f'{len(in_fns)} input data file(s) to regrid')
    for in_fn in in_fns:
        # Output data filename
        if stat_str:
            if missto0:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}_missto0.{stat_str.strip("-")}.nc')
            else:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.{stat_str.strip("-")}.nc')
        else:
            if missto0:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}_missto0.nc')
            else:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.nc')
        # If output file already exists, do not overwrite (unless force=True)
        if out_fn.exists() and not force:
            existing_list.append(out_fn.name)
            continue
        # Regrid using CDO
        if missto0:
            cdo_res = cdo.sellonlatbox(0, 360, -90, 90,  # shift longitudes
                                input=(f'{stat_str} -selname,{variable} -setmisstoc,0 -remap,global_1,{weights_fn}'
                                       f' -setmisstoc,0 -setctomiss,nan {in_fn}'),
                                output=f'{out_fn}')
        else:
            cdo_res = cdo.sellonlatbox(0, 360, -90, 90,  # shift longitudes
                                input=f'{stat_str} -selname,{variable} -remap,global_1,{weights_fn} {in_fn}',
                                output=f'{out_fn}')
        # Check if written
        if out_fn.exists():
            out_list.append(out_fn.name)
        else:
            print(f'Failed to write {out_fn.name}')
    
    # Print summary of existing files skipped
    if len(existing_list) == 1:
        print(f'Skipped existing file: {existing_list[0]}')
    elif len(existing_list) > 1:
        print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
    # Print summary of files written (if points=1)
    if len(out_list) == 1:
        print(f'Written {out_list[0]}')
    elif len(out_list) > 1:
        print(f'Written {len(out_list)} files, inc {out_list[0]}')
    
    return out_list  # exit

## Function to calculate fldmeans (skipping missing data) or glbmeans (replacing missing data with 0) for a given variable, table_id, source-member pair, and experiment - using regridded data produced above as input
Update: glbmean functionality is redundant if using newly implemented missto0 in regrid_using_cdo() above.

In [6]:
def fldstat_using_cdo(fld_or_glbstat='fldmean',
                      in_group='regrid_missto0_yearmean',  # data produced by regrid_using_cdo()
                      variable='zos',
                      table_id='Omon',
                      source_member='ACCESS-CM2_r1i1p1f1',
                      experiment='historical',
                      force=False,  # overwrite previously produced files?
                      ):
    
    print(f'---- {fld_or_glbstat} {in_group} {variable} {table_id} {source_member} {experiment} ----')
    
    # Is fldstat_or_glbstat recognised?
    if fld_or_glbstat not in ['fldmean', 'glbmean']:
        print(f'{fld_or_glbstat} not recognized')
        return None  # exit
    
    # List of pre-existing files skipped (assuming force=False)
    existing_list = []
    # List of output files written
    out_list = []
    
    # Search for available input directories, then identify most appropriate directory
    in_dirs = list(out_base.glob(
        f'{in_group}/{variable}/{source_member}/CMIP6.*.{experiment}.*.{table_id}.{variable}.*'))
    in_dirs = sorted(in_dirs)
    # Ignore directories with no NetCDF files
    for d in in_dirs:
        if len(list(d.glob("*.nc"))) == 0:
            in_dirs.remove(d)
    # Case: no input directories
    if len(in_dirs) == 0:
        print('No suitable input data found')
        return None  # exit (if no input directories)
    # Case: one suitable input directory
    elif len(in_dirs) == 1:
        in_dir = in_dirs[0]
        print(f'1 input dir: {in_dir.name} ({len(list(in_dir.glob("*.nc")))} nc file(s))')
    # Case: more than one suitable input directory
    else:
        print(f'{len(in_dirs)} input dirs:')
        for d in in_dirs:
            print(f'{d.name} ({len(list(d.glob("*.nc")))} nc file(s))')
        # Select most recently dated version
        vmax = 0  # initialise "max" version as 0
        for d in in_dirs:
            vmax = max(vmax, int(d.name[-8:]))
        for d in in_dirs:
            if str(vmax) not in d.name:
                in_dirs.remove(d)
        # After version selection, does only one input dir remain?
        if len(in_dirs) == 1:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (most recent) ')
        else:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name}')
        
    # Identify input data files
    in_fns = sorted(list(in_dir.glob('*.nc')))
    # Output data directory
    out_dir = out_base.joinpath(f'{in_group}_{fld_or_glbstat}/{variable}/{source_member}/{in_dir.name}')
    out_dir.mkdir(parents=True, exist_ok=True)

    # Loop over input data files and apply CDO
    print(f'{len(in_fns)} input data file(s) for which to calculate {fld_or_glbstat}')
    for in_fn in in_fns:
        # Output data filename
        out_fn = out_dir.joinpath(f'{in_fn.stem}.{fld_or_glbstat}.nc')
        # If output file already exists, do not overwrite (unless force=True)
        if out_fn.exists() and not force:
            existing_list.append(out_fn.name)
            continue
        # If fldmean etc, follow default CDO behaviour of skipping missing data
        if fld_or_glbstat[0:3] == 'fld':
            cdo_res = cdo.selname(f'{variable}',
                                  input=f'-{fld_or_glbstat} {in_fn}',
                                  output=f'{out_fn}')
        # But if glbmean etc, treating missing as zero
        elif fld_or_glbstat[0:3] == 'glb':
            cdo_res = cdo.selname(f'{variable}',
                                  input=f'-fld{fld_or_glbstat[3:]} -setmisstoc,0 {in_fn}',
                                  output=f'{out_fn}')
        # Check if written
        if out_fn.exists():
            out_list.append(out_fn.name)
        else:
            print(f'Failed to write {out_fn.name}')
    
    # Print summary of existing files skipped
    if len(existing_list) == 1:
        print(f'Skipped existing file: {existing_list[0]}')
    elif len(existing_list) > 1:
        print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
    # Print summary of files written (if points=1)
    if len(out_list) == 1:
        print(f'Written {out_list[0]}')
    elif len(out_list) > 1:
        print(f'Written {len(out_list)} files, inc {out_list[0]}')
    
    return out_list  # exit

## Apply function(s) to variables and experiments of interest

### Omon data

In [7]:
%%time
# For variables and experiments of interest, apply regridding and fldstat functions
# Loop over variables of interest
table_id = 'Omon'
for variable in ['zostoga', 'thetaoga',  # 1D ocean
                 'zos', 'hfds', 'hfgeou', 'hfcorr']:  # 2D ocean
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest - include piControl for Omon
        for experiment in ['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126']:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    missto0=True,
                                    stat_str='-yearmean',
                                    force=False)
            # fldmean
            temp = fldstat_using_cdo(fld_or_glbstat='fldmean',
                                     in_group='regrid_missto0_yearmean',
                                     variable=variable,
                                     table_id=table_id,
                                     source_member=source_member,
                                     experiment=experiment,
                                     force=False)
#             # glbmean
#             temp = fldstat_using_cdo(fld_or_glbstat='glbmean',
#                                      in_group='regrid_missto0_yearmean',
#                                      variable=variable,
#                                      table_id=table_id,
#                                      source_member=source_member,
#                                      experiment=experiment,
#                                      force=False)
        print('', flush=True)  # empty line between source-member pairs

---- regrid zostoga Omon ACCESS-CM2_r1i1p1f1 piControl missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.piControl.r1i1p1f1.Omon.zostoga.gn.v20191112 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.1d.yearmean.nc
---- fldmean regrid_missto0_yearmean zostoga Omon ACCESS-CM2_r1i1p1f1 piControl ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.piControl.r1i1p1f1.Omon.zostoga.gn.v20191112 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: zostoga_Omon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.1d.yearmean.fldmean.nc
---- regrid zostoga Omon ACCESS-CM2_r1i1p1f1 historical missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Omon.zostoga.gn.v20191108 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_ACCESS-CM

---- regrid zostoga Omon BCC-CSM2-MR_r1i1p1f1 piControl missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.BCC.BCC-CSM2-MR.piControl.r1i1p1f1.Omon.zostoga.gn.v20181015 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_BCC-CSM2-MR_piControl_r1i1p1f1_gn_185001-244912.1d.yearmean.nc
---- fldmean regrid_missto0_yearmean zostoga Omon BCC-CSM2-MR_r1i1p1f1 piControl ----
1 input dir: CMIP6.CMIP.BCC.BCC-CSM2-MR.piControl.r1i1p1f1.Omon.zostoga.gn.v20181015 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: zostoga_Omon_BCC-CSM2-MR_piControl_r1i1p1f1_gn_185001-244912.1d.yearmean.fldmean.nc
---- regrid zostoga Omon BCC-CSM2-MR_r1i1p1f1 historical missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.BCC.BCC-CSM2-MR.historical.r1i1p1f1.Omon.zostoga.gn.v20181126 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_BCC-CSM2-MR_historical_r1i1p

---- regrid zostoga Omon CMCC-ESM2_r1i1p1f1 piControl missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.CMCC.CMCC-ESM2.piControl.r1i1p1f1.Omon.zostoga.gn.v20210304 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CMCC-ESM2_piControl_r1i1p1f1_gn_185001-234912.1d.yearmean.nc
---- fldmean regrid_missto0_yearmean zostoga Omon CMCC-ESM2_r1i1p1f1 piControl ----
1 input dir: CMIP6.CMIP.CMCC.CMCC-ESM2.piControl.r1i1p1f1.Omon.zostoga.gn.v20210304 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: zostoga_Omon_CMCC-ESM2_piControl_r1i1p1f1_gn_185001-234912.1d.yearmean.fldmean.nc
---- regrid zostoga Omon CMCC-ESM2_r1i1p1f1 historical missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.CMCC.CMCC-ESM2.historical.r1i1p1f1.Omon.zostoga.gn.v20210114 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CMCC-ESM2_historical_r1i1p1f1_gn_185001-2

Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CNRM-CM6-1_historical_r1i1p1f2_gn_185001-201412.1d.yearmean.nc
---- fldmean regrid_missto0_yearmean zostoga Omon CNRM-CM6-1_r1i1p1f2 historical ----
1 input dir: CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.historical.r1i1p1f2.Omon.zostoga.gn.v20180917 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: zostoga_Omon_CNRM-CM6-1_historical_r1i1p1f2_gn_185001-201412.1d.yearmean.fldmean.nc
---- regrid zostoga Omon CNRM-CM6-1_r1i1p1f2 ssp585 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1.ssp585.r1i1p1f2.Omon.zostoga.gn.v20190219 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CNRM-CM6-1_ssp585_r1i1p1f2_gn_201501-210012.1d.yearmean.nc
---- fldmean regrid_missto0_yearmean zostoga Omon CNRM-CM6-1_r1i1p1f2 ssp585 ----
1 input dir: CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1.s

Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CanESM5-CanOE_ssp585_r1i1p2f1_gn_201501-210012.1d.yearmean.nc
---- fldmean regrid_missto0_yearmean zostoga Omon CanESM5-CanOE_r1i1p2f1 ssp585 ----
1 input dir: CMIP6.ScenarioMIP.CCCma.CanESM5-CanOE.ssp585.r1i1p2f1.Omon.zostoga.gn.v20190429 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: zostoga_Omon_CanESM5-CanOE_ssp585_r1i1p2f1_gn_201501-210012.1d.yearmean.fldmean.nc
---- regrid zostoga Omon CanESM5-CanOE_r1i1p2f1 ssp370 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CCCma.CanESM5-CanOE.ssp370.r1i1p2f1.Omon.zostoga.gn.v20190429 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CanESM5-CanOE_ssp370_r1i1p2f1_gn_201501-210012.1d.yearmean.nc
---- fldmean regrid_missto0_yearmean zostoga Omon CanESM5-CanOE_r1i1p2f1 ssp370 ----
1 input dir: CMIP6.ScenarioMIP.CCCma.CanESM5-CanOE.ssp3

...

Input data grid type is curvilinear (points=118800 (360x330))
Trying genbic
hfds_Omon_UKESM1-0-LL_ssp126_r1i1p1f2_gn_201501-204912.genbic_weights.nc already exists
2 input data file(s) to regrid
Skipped 2 existing files, inc hfds_Omon_UKESM1-0-LL_ssp126_r1i1p1f2_gn_201501-204912.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean hfds Omon UKESM1-0-LL_r1i1p1f2 ssp126 ----
1 input dir: CMIP6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp126.r1i1p1f2.Omon.hfds.gn.v20190708 (2 nc file(s))
2 input data file(s) for which to calculate fldmean
Skipped 2 existing files, inc hfds_Omon_UKESM1-0-LL_ssp126_r1i1p1f2_gn_201501-204912.bic_missto0.yearmean.fldmean.nc

---- regrid hfgeou Omon EC-Earth3-AerChem_r1i1p1f1 piControl missto0=True -yearmean ----
No suitable input data found
---- fldmean regrid_missto0_yearmean hfgeou Omon EC-Earth3-AerChem_r1i1p1f1 piControl ----
No suitable input data found
---- regrid hfgeou Omon EC-Earth3-AerChem_r1i1p1f1 historical missto0=True -yearmean ----
No suitable inpu

Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfgeou_Omon_EC-Earth3-Veg-LR_ssp126_r1i1p1f1_gn_201501-201512.genbic_weights.nc already exists
86 input data file(s) to regrid
Skipped 86 existing files, inc hfgeou_Omon_EC-Earth3-Veg-LR_ssp126_r1i1p1f1_gn_201501-201512.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean hfgeou Omon EC-Earth3-Veg-LR_r1i1p1f1 ssp126 ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.ssp126.r1i1p1f1.Omon.hfgeou.gn.v20201201 (86 nc file(s))
86 input data file(s) for which to calculate fldmean
Skipped 86 existing files, inc hfgeou_Omon_EC-Earth3-Veg-LR_ssp126_r1i1p1f1_gn_201501-201512.bic_missto0.yearmean.fldmean.nc

---- regrid hfgeou Omon EC-Earth3-Veg_r1i1p1f1 piControl missto0=True -yearmean ----
No suitable input data found
---- fldmean regrid_missto0_yearmean hfgeou Omon EC-Earth3-Veg_r1i1p1f1 piControl ----
No suitable input data found
---- regrid hfgeou Omon EC-Earth3-Veg_r1i1p1f1 historical m

Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfgeou_Omon_EC-Earth3_ssp245_r1i1p1f1_gn_201501-201512.genbic_weights.nc already exists
86 input data file(s) to regrid
Skipped 86 existing files, inc hfgeou_Omon_EC-Earth3_ssp245_r1i1p1f1_gn_201501-201512.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean hfgeou Omon EC-Earth3_r1i1p1f1 ssp245 ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.r1i1p1f1.Omon.hfgeou.gn.v20200918 (86 nc file(s))
86 input data file(s) for which to calculate fldmean
Skipped 86 existing files, inc hfgeou_Omon_EC-Earth3_ssp245_r1i1p1f1_gn_201501-201512.bic_missto0.yearmean.fldmean.nc
---- regrid hfgeou Omon EC-Earth3_r1i1p1f1 ssp126 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp126.r1i1p1f1.Omon.hfgeou.gn.v20200918 (86 nc file(s))
Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfgeou_Omon_EC-Earth3_ssp126_r1i1p1f1_gn_201501-20151

Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfcorr_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.genbic_weights.nc already exists
165 input data file(s) to regrid
Skipped 165 existing files, inc hfcorr_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean hfcorr Omon EC-Earth3-CC_r1i1p1f1 historical ----
1 input dir: CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-CC.historical.r1i1p1f1.Omon.hfcorr.gn.v20210113 (165 nc file(s))
165 input data file(s) for which to calculate fldmean
Skipped 165 existing files, inc hfcorr_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.bic_missto0.yearmean.fldmean.nc
---- regrid hfcorr Omon EC-Earth3-CC_r1i1p1f1 ssp585 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp585.r1i1p1f1.Omon.hfcorr.gn.v20210113 (86 nc file(s))
Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfcorr_Omon_EC

---- regrid hfcorr Omon MRI-ESM2-0_r1i1p1f1 piControl missto0=True -yearmean ----
2 input dirs:
CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Omon.hfcorr.gn.v20210311 (1 nc file(s))
CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Omon.hfcorr.gr.v20191224 (1 nc file(s))
Using CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Omon.hfcorr.gn.v20210311 (most recent) 
Input data grid type is curvilinear (points=130680 (360x363))
Trying genbic
hfcorr_Omon_MRI-ESM2-0_piControl_r1i1p1f1_gn_185001-255012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: hfcorr_Omon_MRI-ESM2-0_piControl_r1i1p1f1_gn_185001-255012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean hfcorr Omon MRI-ESM2-0_r1i1p1f1 piControl ----
1 input dir: CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Omon.hfcorr.gn.v20210311 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: hfcorr_Omon_MRI-ESM2-0_piControl_r1i1p1f1_gn_185001-255012.bic_missto0.yearmean.f

### Annual means (yearmean) of regridded Amon data

In [8]:
%%time
# For variables and experiments of interest, apply regridding and fldstat functions
# Loop over variables of interest
table_id = 'Amon'
for variable in ['rlut', 'rsdt', 'rsut', 'tas']:
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest - include piControl for Omon
        for experiment in ['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126']:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    missto0=True,
                                    stat_str='-yearmean',
                                    force=False)
            # fldmean
            temp = fldstat_using_cdo(fld_or_glbstat='fldmean',
                                     in_group='regrid_missto0_yearmean',
                                     variable=variable,
                                     table_id=table_id,
                                     source_member=source_member,
                                     experiment=experiment,
                                     force=False)
#             # glbmean
#             temp = fldstat_using_cdo(fld_or_glbstat='glbmean',
#                                      in_group='regrid_missto0_yearmean',
#                                      variable=variable,
#                                      table_id=table_id,
#                                      source_member=source_member,
#                                      experiment=experiment,
#                                      force=False)
        print('', flush=True)  # empty line between source-member pairs

---- regrid rlut Amon ACCESS-CM2_r1i1p1f1 piControl missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.piControl.r1i1p1f1.Amon.rlut.gn.v20191112 (1 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying genbic
rlut_Amon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean rlut Amon ACCESS-CM2_r1i1p1f1 piControl ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.piControl.r1i1p1f1.Amon.rlut.gn.v20191112 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: rlut_Amon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.bic_missto0.yearmean.fldmean.nc
---- regrid rlut Amon ACCESS-CM2_r1i1p1f1 historical missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Amon.rlut.gn.v201911

Input data grid type is lonlat (points=27840 (192x145))
Trying genbic
rlut_Amon_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean rlut Amon ACCESS-ESM1-5_r1i1p1f1 ssp245 ----
1 input dir: CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp245.r1i1p1f1.Amon.rlut.gn.v20191115 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: rlut_Amon_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.fldmean.nc
---- regrid rlut Amon ACCESS-ESM1-5_r1i1p1f1 ssp126 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CSIRO.ACCESS-ESM1-5.ssp126.r1i1p1f1.Amon.rlut.gn.v20210318 (2 nc file(s))
Input data grid type is lonlat (points=27840 (192x145))
Trying genbic
rlut_Amon_ACCESS-ESM1-5_ssp126_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
2 input d

Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
rlut_Amon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean rlut Amon BCC-CSM2-MR_r1i1p1f1 ssp585 ----
1 input dir: CMIP6.ScenarioMIP.BCC.BCC-CSM2-MR.ssp585.r1i1p1f1.Amon.rlut.gn.v20190314 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: rlut_Amon_BCC-CSM2-MR_ssp585_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.fldmean.nc
---- regrid rlut Amon BCC-CSM2-MR_r1i1p1f1 ssp370 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.BCC.BCC-CSM2-MR.ssp370.r1i1p1f1.Amon.rlut.gn.v20190314 (1 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
rlut_Amon_BCC-CSM2-MR_ssp370_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data fi

Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
rlut_Amon_CAMS-CSM1-0_ssp370_r1i1p1f1_gn_201501-209912.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_CAMS-CSM1-0_ssp370_r1i1p1f1_gn_201501-209912.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean rlut Amon CAMS-CSM1-0_r1i1p1f1 ssp370 ----
1 input dir: CMIP6.ScenarioMIP.CAMS.CAMS-CSM1-0.ssp370.r1i1p1f1.Amon.rlut.gn.v20190708 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: rlut_Amon_CAMS-CSM1-0_ssp370_r1i1p1f1_gn_201501-209912.bic_missto0.yearmean.fldmean.nc
---- regrid rlut Amon CAMS-CSM1-0_r1i1p1f1 ssp245 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CAMS.CAMS-CSM1-0.ssp245.r1i1p1f1.Amon.rlut.gn.v20190708 (1 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
rlut_Amon_CAMS-CSM1-0_ssp245_r1i1p1f1_gn_201501-209912.genbic_weights.nc already exists
1 input data 

Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
rlut_Amon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
5 input data file(s) to regrid
Skipped 5 existing files, inc rlut_Amon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean rlut Amon CESM2-WACCM_r1i1p1f1 ssp585 ----
1 input dir: CMIP6.ScenarioMIP.NCAR.CESM2-WACCM.ssp585.r1i1p1f1.Amon.rlut.gn.v20200702 (5 nc file(s))
5 input data file(s) for which to calculate fldmean
Skipped 5 existing files, inc rlut_Amon_CESM2-WACCM_ssp585_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.fldmean.nc
---- regrid rlut Amon CESM2-WACCM_r1i1p1f1 ssp370 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NCAR.CESM2-WACCM.ssp370.r1i1p1f1.Amon.rlut.gn.v20190815 (2 nc file(s))
Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
rlut_Amon_CESM2-WACCM_ssp370_r1i1p1f1_gn_201501-206412.genbic_weights.nc already exists
2 input data 

...

Input data grid type is gaussian (points=73728 (384x192)  F96)
Trying genbic
tas_Amon_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_201501-201912.genbic_weights.nc already exists
18 input data file(s) to regrid
Skipped 18 existing files, inc tas_Amon_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_201501-201912.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean tas Amon MPI-ESM1-2-HR_r1i1p1f1 ssp585 ----
1 input dir: CMIP6.ScenarioMIP.DKRZ.MPI-ESM1-2-HR.ssp585.r1i1p1f1.Amon.tas.gn.v20190710 (18 nc file(s))
18 input data file(s) for which to calculate fldmean
Skipped 18 existing files, inc tas_Amon_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_201501-201912.bic_missto0.yearmean.fldmean.nc
---- regrid tas Amon MPI-ESM1-2-HR_r1i1p1f1 ssp370 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.DKRZ.MPI-ESM1-2-HR.ssp370.r1i1p1f1.Amon.tas.gn.v20190710 (18 nc file(s))
Input data grid type is gaussian (points=73728 (384x192)  F96)
Trying genbic
tas_Amon_MPI-ESM1-2-HR_ssp370_r1i1p1f1_gn_201501-201912.genbic_weights.nc 

---- regrid tas Amon MRI-ESM2-0_r1i1p1f1 piControl missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Amon.tas.gn.v20190222 (1 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
tas_Amon_MRI-ESM2-0_piControl_r1i1p1f1_gn_185001-255012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_MRI-ESM2-0_piControl_r1i1p1f1_gn_185001-255012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean tas Amon MRI-ESM2-0_r1i1p1f1 piControl ----
1 input dir: CMIP6.CMIP.MRI.MRI-ESM2-0.piControl.r1i1p1f1.Amon.tas.gn.v20190222 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: tas_Amon_MRI-ESM2-0_piControl_r1i1p1f1_gn_185001-255012.bic_missto0.yearmean.fldmean.nc
---- regrid tas Amon MRI-ESM2-0_r1i1p1f1 historical missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r1i1p1f1.Amon.tas.gn.v20190222 (1 nc file(s))
Input data 

Input data grid type is gaussian (points=18432 (192x96)  F48)
Trying genbic
tas_Amon_NESM3_ssp126_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_NESM3_ssp126_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean tas Amon NESM3_r1i1p1f1 ssp126 ----
1 input dir: CMIP6.ScenarioMIP.NUIST.NESM3.ssp126.r1i1p1f1.Amon.tas.gn.v20190731 (1 nc file(s))
1 input data file(s) for which to calculate fldmean
Skipped existing file: tas_Amon_NESM3_ssp126_r1i1p1f1_gn_201501-210012.bic_missto0.yearmean.fldmean.nc

---- regrid tas Amon NorESM2-LM_r1i1p1f1 piControl missto0=True -yearmean ----
1 input dir: CMIP6.CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Amon.tas.gn.v20210118 (51 nc file(s))
Input data grid type is lonlat (points=13824 (144x96))
Trying genbic
tas_Amon_NorESM2-LM_piControl_r1i1p1f1_gn_160001-160912.genbic_weights.nc already exists
51 input data file(s) to regrid
Skipped 51 existing files, 

Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
tas_Amon_NorESM2-MM_ssp245_r1i1p1f1_gn_201501-202012.genbic_weights.nc already exists
9 input data file(s) to regrid
Skipped 9 existing files, inc tas_Amon_NorESM2-MM_ssp245_r1i1p1f1_gn_201501-202012.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean tas Amon NorESM2-MM_r1i1p1f1 ssp245 ----
1 input dir: CMIP6.ScenarioMIP.NCC.NorESM2-MM.ssp245.r1i1p1f1.Amon.tas.gn.v20191108 (9 nc file(s))
9 input data file(s) for which to calculate fldmean
Skipped 9 existing files, inc tas_Amon_NorESM2-MM_ssp245_r1i1p1f1_gn_201501-202012.bic_missto0.yearmean.fldmean.nc
---- regrid tas Amon NorESM2-MM_r1i1p1f1 ssp126 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NCC.NorESM2-MM.ssp126.r1i1p1f1.Amon.tas.gn.v20191108 (9 nc file(s))
Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
tas_Amon_NorESM2-MM_ssp126_r1i1p1f1_gn_201501-202012.genbic_weights.nc already exists
9 input data file(s) to regrid


Input data grid type is lonlat (points=27648 (192x144))
Trying genbic
tas_Amon_UKESM1-0-LL_ssp370_r1i1p1f2_gn_201501-204912.genbic_weights.nc already exists
2 input data file(s) to regrid
Skipped 2 existing files, inc tas_Amon_UKESM1-0-LL_ssp370_r1i1p1f2_gn_201501-204912.bic_missto0.yearmean.nc
---- fldmean regrid_missto0_yearmean tas Amon UKESM1-0-LL_r1i1p1f2 ssp370 ----
1 input dir: CMIP6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp370.r1i1p1f2.Amon.tas.gn.v20190510 (2 nc file(s))
2 input data file(s) for which to calculate fldmean
Skipped 2 existing files, inc tas_Amon_UKESM1-0-LL_ssp370_r1i1p1f2_gn_201501-204912.bic_missto0.yearmean.fldmean.nc
---- regrid tas Amon UKESM1-0-LL_r1i1p1f2 ssp245 missto0=True -yearmean ----
1 input dir: CMIP6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp245.r1i1p1f2.Amon.tas.gn.v20190507 (2 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying genbic
tas_Amon_UKESM1-0-LL_ssp245_r1i1p1f2_gn_201501-204912.genbic_weights.nc already exists
2 input data file(s) 

In [9]:
! date

Mon Apr 11 09:38:39 +08 2022
